## Calcluate the dot product between vectore using loop and list, the complixty is high, O(n^2d)

In [25]:
def compute_dot_product(matrix):
  n=len(matrix) #how many vector
  dotProductMatrix = [[0 for _ in range(n)] for _ in range(n)] # Create dot product matrix to save the result inside it
  for i in range(n):
    for j in range(i,n): # loop from i to n to handle the redudent on the matrix
      dotProduct=0
      for k in range(n):
        dotProduct=matrix[i][k]*matrix[j][k]
      #Save it diagonally
      dotProductMatrix[i][j]=dotProduct 
      dotProductMatrix[j][i]=dotProduct
  return dotProductMatrix

matrix=[[1.2,2.1,3.5],[2.1,3.3,4.5]]
compute_dot_product(matrix)

[[4.41, 6.93], [6.93, 10.889999999999999]]

In [26]:
def find_nearest_differences(matrix):
    n = len(matrix)  # Number of rows
    nearest = [[0 for _ in range(len(matrix[0]))] for _ in range(n)]  # Initialize the result matrix
    for i in range(n): #loop on matrix
        m = len(matrix[i])
        for j in range(m):
            minDiff = float('inf')  # Initialize minimum difference to infinity
            for k in range(m):
                if j != k:
                    diff = abs(matrix[i][j] - matrix[i][k]) # get the difference between the number and the vector
                    minDiff = min(minDiff, diff)
            nearest[i][j] = minDiff

    return nearest

matrix=[[1.2,2.1,3.5],[2.1,3.3,4.5]]
find_nearest_differences(matrix)


[[0.9000000000000001, 0.9000000000000001, 1.4],
 [1.1999999999999997, 1.1999999999999997, 1.2000000000000002]]

## Make enhancment on the code using numpy and minimize the using of the list

In [27]:
import numpy as np
def compute_dot_product_numpy(matrix):
    matrixNumpy = np.array(matrix)
    dot_product_matrix = np.dot(matrixNumpy, matrixNumpy.T) #make dot product using numpy dot
    return dot_product_matrix

In [28]:
def find_nearest_differences_numpy(matrix):
    matrixNumpy = np.array(matrix)
    r, c = matrixNumpy.shape
    nearest = np.zeros((r, c)) # initilize matrix zeros for saving the values of the difference

    for i in range(r):
        rowRepeated = matrixNumpy[i, :].reshape(c, 1)
        differences = np.abs(rowRepeated - matrixNumpy[i, :]) #calculate the difference
        np.fill_diagonal(differences, np.inf) # fill digonal to infinity to correct the min calculation
        nearest[i, :] = np.min(differences, axis=1)   # Find the minimum difference for each element
    return nearest

In [29]:
matrix=[[1.2,2.1,3.5],[2.1,3.3,4.5]]
find_nearest_differences(matrix)
find_nearest_differences_numpy(matrix)

array([[0.9, 0.9, 1.4],
       [1.2, 1.2, 1.2]])

## make the partial dot product using parallel processing

In [48]:
def compute_partial_dot_product_chunck(matrixChunk, matrix):
    return np.dot(matrixChunk, matrix.T)

def compute_dot_product_parallel(matrix, num_processes=4):
    matrixNumpy = np.array(matrix)
    # split chunks for processing
    rowChunks = np.array_split(matrixNumpy, num_processes)

    with Pool(num_processes) as pool:
        # Compute dot product for each chunk
        results = pool.starmap(compute_partial_dot_product_chunck, [(chunk, matrixNumpy) for chunk in rowChunks])

    return np.vstack(results)

In [49]:
def compute_nearest_differences_subset(args):
    matrixNumpy, rows = args
    r, c = matrixNumpy.shape
    nearest = np.zeros((len(rows), c)) # initilize matrix zeros for saving the values of the difference


    for idx, i in enumerate(rows):
        rowRepeated = matrixNumpy[i, :].reshape(c, 1)
        differences = np.abs(rowRepeated - matrixNumpy[i, :])
        np.fill_diagonal(differences, np.inf) # fill diagonal to infinity
        nearest[idx, :] = np.min(differences, axis=1) # Find minimum difference for each element

    return nearest

def find_nearest_differences_parallel(matrix, num_processes=4):
    matrixNumpy = np.array(matrix)
    r, c = matrixNumpy.shape

    # Ensure that the matrix is two-dimensional
    if len(matrixNumpy.shape) != 2:
        raise ValueError("Matrix must be two-dimensional")

    rows_per_process = np.array_split(range(r), num_processes)

    with Pool(num_processes) as pool:
        results = pool.map(compute_nearest_differences_subset, [(matrixNumpy, rows) for rows in rows_per_process])

    nearest = np.vstack(results) # return it in vstack

    return nearest

In [50]:
matrix= np.random.rand(200, 200) * 5 #create the matrix 200 * 200 to check the different of using the above functions

In [51]:
matrix.shape


(200, 200)

In [52]:
import time
startTimeDot = time.time()
compute_dot_product(matrix)
endTimeDot = time.time()-startTimeDot

startTimeNumpy = time.time()
compute_dot_product_numpy(matrix)
endTimeDotNumpy = time.time()-startTimeNumpy

startTimeParallel = time.time()
compute_dot_product_parallel(matrix)
endTimeDotParallel = time.time()-startTimeParallel

endTimeDot,endTimeDotNumpy,endTimeDotParallel

(2.1329360008239746, 0.001352071762084961, 0.0688941478729248)

In [54]:
import time
startTimeDot = time.time()
find_nearest_differences(matrix)
endTimeFind = time.time()-startTimeDot

startTimeNumpy = time.time()
find_nearest_differences_numpy(matrix)
endTimeFindNumpy = time.time()-startTimeNumpy

startTimeParallel = time.time()
find_nearest_differences_parallel(matrix)
endTimeFindParallel = time.time()-startTimeParallel

endTimeFind,endTimeFindNumpy,endTimeFindParallel

(5.857277154922485, 0.021490097045898438, 0.040221452713012695)

The function to compute the dot product between each pair of vectors is related to UN-AVOIDS/NCDF as its comparing the similarity or dissimilarity between vectors
For the second point as it involves comparing values within a matrix (Dataset), The function to find the nearest value differences within each row of the array can be useful in identifying outliers or anomalies within the dataset. 

Thanks